# Lab 1 Part I: MNIST 2D CNN

## Imports

In [2]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

import wandb
from wandb.keras import WandbMetricsLogger

## Load and format MNIST dataset

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255
x_test  = x_test.astype('float32')  / 255
x_train = x_train.reshape((60000, 28, 28, 1)) # 'channels_last' format
x_test  = x_test.reshape((10000, 28, 28, 1)) # 'channels_last' format
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test,  10)

## Build model

In [4]:
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(filters=2, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(units=10, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


# log NN details
wandb.init(
    project="mnist_v2",
    name="Kernel Size: 5x5 1 Pooling Layer | 2x2DConv (32 => 64) | 3 Epochs | Run 3",
    config=model.get_config()
)

model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])



Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 2)         20        
                                                                 
 flatten (Flatten)           (None, 1352)              0         
                                                                 
 dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,550
Trainable params: 13,550
Non-trainable params: 0
_________________________________________________________________


2023-03-03 17:21:34.800679: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-03 17:21:34.800870: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
wandb: Currently logged in as: juzay (juzay_and_co). Use `wandb login --relogin` to force relogin


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 2)         20        
                                                                 
 flatten (Flatten)           (None, 1352)              0         
                                                                 
 dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,550
Trainable params: 13,550
Non-trainable params: 0
_________________________________________________________________


## Train model

In [8]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test), callbacks=[
        WandbMetricsLogger(log_freq=5)
    ])

Epoch 1/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1315 - categorical_accuracy: 0.9613 - val_loss: 0.1266 - val_categorical_accuracy: 0.9629
Epoch 2/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1229 - categorical_accuracy: 0.9638 - val_loss: 0.1310 - val_categorical_accuracy: 0.9610
Epoch 3/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1167 - categorical_accuracy: 0.9650 - val_loss: 0.1277 - val_categorical_accuracy: 0.9626


wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


## Evaluate model

In [6]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

313/313 - 2s - loss: 0.1406 - categorical_accuracy: 0.9599 - 2s/epoch - 7ms/step
 84/313 [=======>......................] - ETA: 0s

2023-03-03 17:22:38.186275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 2ms/step
tf.Tensor(
[[ 967    0    3    1    1    3    1    1    1    2]
 [   0 1122    3    0    1    0    2    0    5    2]
 [   1    6  989    5    9    0    1   10    9    2]
 [   1    0   12  961    0   13    0    8    9    6]
 [   1    1    4    0  961    0    0    1    1   13]
 [   4    1    1   10    0  861    3    3    3    6]
 [  13    4    8    0    7   17  903    0    6    0]
 [   1    5   19    8    3    0    0  969    0   23]
 [   7    4   14    8    8   10    2    6  899   16]
 [   6    6    3    6    9    3    0    7    2  967]], shape=(10, 10), dtype=int32)


## Save trained model

In [7]:
model.save('lab1_part1_mnist.h5')